In [510]:
import numpy as np
import pandas as pd
import csv
from io import StringIO # import StringIO module
import glob
import os

In [511]:
def get_df_5_partidos(link):

    with open(link, "r") as f:
        text = f.read() # read the file as a string

    # replace the substring in the whole string
    text = text.replace('COMPROMISO,', 'COMPROMISO')
    text = text.replace('COMPROMISO.', 'COMPROMISO')

    # convert the string to a list of lines
    rows = text.splitlines()

    start = [i for i, s in enumerate(rows) if "Código ,Nombre" in s][0]

    data = rows[start:]

    # join the data lines into a single string
    data_str = '\n'.join(data)

    # convert the data string to a StringIO object
    data_io = StringIO(data_str)

    # read the data into a dataframe with comma separator
    df = pd.read_csv(data_io, sep=',',dtype=str,  names=['col1', 'col2', 'col3', 'col4', 'col5', 'col6','col7', 'col8', 'col9','col10'], quoting=csv.QUOTE_NONE)
    df = df.iloc[1:25].copy()
    lenght = len(df.columns)
    result = df.iloc[:,2].fillna('')

    for i in range(3,lenght):
        result = result + df.iloc[:,i].fillna('')
    eliminate_special_characters = {' ':'', '\.':'','/':'1','o':'0','O':'0', ':':'','C':'0','-':'', 'X':'','a':'9',
                                    '\(':'','\)':'','I':'1', 'U':'0', 'G':'6','D':'0','S':'5','B':'8','E':'8','A':'4',
                                    'b':'6','s':'5','\#':'4', '\+':'7', 'Q':'0', "\\\\":'1', 'H':'4', '&':'8', '\>':'',
                                    '\$':'5', 'x':'','\?':'3', 'J':'','T':'7', 'n':'6', 't':'7','\<':'5', '\}':'3',
                                    'Z':'2'}
    df["votos_ocr"] = result.replace(eliminate_special_characters, regex=True)
    df = df[["col1","votos_ocr"]].copy()
    df.columns = ["codigo","votos_ocr"]
    #df_final = df[df["codigo"].str.contains("23") |df["codigo"].str.contains("15")|df["codigo"].str.contains("41")|df["codigo"].str.contains("88")|df["codigo"].str.contains("20") ].copy()

    df_final = df[df["codigo"].str.contains("41") |df["codigo"].str.contains("20") ].copy()

    df_final["codigo"] = df_final["codigo"].str.rstrip()
    change_numebers = {"23":"TODOS","15":"UNE","41":"VALOR UNIONISTA","88":"FOPPA X LA CIUDAD","20":"CREO"}
    df_final["codigo"] = df_final["codigo"].replace(change_numebers)

    return df_final

In [512]:
directorio_de_archivos = glob.glob("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/final_data/elecciones_2023/corporacion_municipal/distrito_central_csv/*.*")

In [513]:
cantidad_errores = 0
df_general_ocr = pd.DataFrame()
for archivo in directorio_de_archivos:
    filename = os.path.basename(archivo).split(".")[0]
    name_separated = filename.split("_")
    centro = name_separated[2]
    mesa = name_separated[4]

    try:
        df_individual = get_df_5_partidos(archivo)
        df_individual["centro"] = centro
        df_individual["mesa"] = mesa
        df_individual["centro"] = df_individual["centro"].astype(int)
        df_individual["mesa"] = df_individual["mesa"].astype(int)
        df_general_ocr = pd.concat([df_general_ocr,df_individual])
    except:
        print(archivo)
        cantidad_errores = cantidad_errores + 1

In [514]:
cantidad_errores

0

In [515]:
df_general_ocr

,codigo,votos_ocr,centro,mesa
3,VALOR UNIONISTA,64,53,693
24,CREO,039,53,693
3,VALOR UNIONISTA,815,72,933
24,CREO,058,72,933
3,VALOR UNIONISTA,41,111,1512
...,...,...,...,...
24,CREO,051,81,1034
3,VALOR UNIONISTA,076,70,892
24,CREO,064,70,892
3,VALOR UNIONISTA,18,29,366


# Datos del TSE del sistema TREP

In [516]:
df_complete_trep = pd.read_csv("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/data_sistema_trep/4-CORPORACIÓN/gtm2023_e4d00m01.csv", sep=",",skiprows=4)
df_complete_trep = df_complete_trep.iloc[:,:31].copy().drop(columns=["DEPARTAMENTO", "ID_DEPARTAMENTO", "MUNICIPIO", "ID_MUNICIPIO", "PADRÓN"])
df_complete_trep = pd.melt(df_complete_trep, id_vars=["CENTRO_DE_VOTACIÓN", "MESA"], value_vars=df_complete_trep.columns[2:], var_name="codigo", value_name="votos_trep")

df_complete_trep["votos_trep"] = df_complete_trep["votos_trep"].replace("--", 0)
df_complete_trep["votos_trep"] = df_complete_trep["votos_trep"].astype(int)
df_complete_trep.columns = ['centro', 'mesa', 'codigo', 'votos_trep']
df_complete_trep

,centro,mesa,codigo,votos_trep
0,1,1,TODOS,9
1,1,2,TODOS,14
2,1,3,TODOS,10
3,1,4,TODOS,9
4,2,5,TODOS,7
...,...,...,...,...
48547,152,2019,CREO,2
48548,152,2020,CREO,1
48549,152,2021,CREO,2
48550,152,2022,CREO,4


# Merge

In [579]:
def check_votos(row):
  # convert votos_ocr column to int if possible, otherwise return "Incorrect"
  try:
    row["votos_ocr"] = int(row["votos_ocr"])
  except ValueError:
    return "Incorrect"

  # compare votos_ocr and votos_trep columns and return "correct" if they match, otherwise return "Incorrect"
  if row["votos_ocr"] == row["votos_trep"]:
    return "correct"
  else:
    return "Incorrect"

def change_votos(row):
  # convert votos_ocr column to int if possible, otherwise return "Incorrect"
  try:
    row["votos_ocr"] = int(row["votos_ocr"])
  except ValueError:
    return np.NaN

  # compare votos_ocr and votos_trep columns and return "correct" if they match, otherwise return "Incorrect"
  if row["votos_ocr"] == row["votos_trep"]:
    return row["votos_trep"]
  else:
    return np.NaN


In [589]:
df_completo_con_ocr_y_trep = pd.merge(df_general_ocr, df_complete_trep, on=["centro", "mesa", "codigo"],how='left')
df_completo_con_ocr_y_trep['votos_trep'] = df_completo_con_ocr_y_trep['votos_trep']

df_completo_con_ocr_y_trep["result"] = df_completo_con_ocr_y_trep.apply(check_votos, axis=1)
df_completo_con_ocr_y_trep["votos_ocr_correcto"] = df_completo_con_ocr_y_trep.apply(change_votos, axis=1)


df_completo_con_ocr_y_trep.loc[df_completo_con_ocr_y_trep["result"] == "Incorrect", "votos_ocr"] = df_completo_con_ocr_y_trep.loc[df_completo_con_ocr_y_trep["result"] == "Incorrect", "votos_ocr"].replace({'1':''},regex=True)

df_completo_con_ocr_y_trep["result"] = df_completo_con_ocr_y_trep.apply(check_votos, axis=1)
df_completo_con_ocr_y_trep["votos_ocr_correcto"] = df_completo_con_ocr_y_trep.apply(change_votos, axis=1)

df_completo_con_ocr_y_trep.sort_values(by="mesa").reset_index(drop=True,inplace=True)
df_completo_con_ocr_y_trep = df_completo_con_ocr_y_trep[['centro', 'mesa','codigo', 'result', 'votos_trep','votos_ocr','votos_ocr_correcto']].copy()

In [590]:
df_completo_con_ocr_y_trep["result"].value_counts()

correct      3534
Incorrect     494
Name: result, dtype: int64

In [591]:
df_completo_con_ocr_y_trep.to_excel('/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/data_analisis/errores_corporacion_municipal_distrito_central.xlsx',index=False)